In [7]:
import os
import pathlib
import pprint

import itertools

import src.utils.json

---

# Stage 1 ConvNeXt V2 Tiny Fusion Head

In [8]:
exp_params_template =\
{
    "experiment_name": "ret_train_old_smpl/convnext_v2_t_s1/",
    "settings": {
        "device_idxs": [2, 4, 5, 6],
        "backbone": {
            "class": "ConvNeXtV2TinyMultilevelBackbone",
            "img_size": 224
        },
        "head": {
            "class": "RetrievalHeadPyramidBottomUpInstantSimple",
            "in_feat_idxs": [0, 1, 2, 3],
            "emb_size": 1024
        },
        "stage_1": {
            "max_epochs": 5,
            "eval_epoch_0": False,
            "optimizer": {
                "class": "SGD",
                "lr": 5.62e-5,
                "momentum": 0.9
            },
            "scheduler": {
                "class": "ExponentialLR",
                "gamma": 0.95
            },
            "early_stopper": {
                "patience": 5,
                "min_delta": 0
            },
            "data_loading": {
                "batch_size": 32,
                "num_workers": 4,
                "grad_acc_iters": 1
            },
            "autocast": {
                "enabled": False
            },
            "data_gradual_inc": {
                "enabled": False
            },
            "ctsrbm_dataset": {
                "neg_img_filename_list_id": "test_old",
                "train_random_seed": 42,
                "cutdown_ratio": 1
            }
        }
    }
}

head_dicts = {
    "head_000": {
        "class": "RetrievalHeadPyramidBottomUpInstantSimple",
        "in_feat_idxs": [0, 1, 2, 3],
        "emb_size": 1024
    },
    "head_001": {
        "class": "RetrievalHeadPyramidBottomUpProgressiveSimple",
        "in_feat_idxs": [0, 1, 2, 3],
        "emb_sizes": [820, 956, 1024]
    },
    "head_002": {
        "class": "RetrievalHeadPyramidBottomUpProgressiveSimple",
        "in_feat_idxs": [0, 1, 2, 3],
        "emb_sizes": [512, 768, 1024]
    },
    "head_003": {
        "class": "RetrievalHeadPyramidBottomUpInstantConv",
        "in_feat_idxs": [0, 1, 2, 3],
        "emb_size": 1024,
        "conv_par_perc": 0
    },
    "head_004": {
        "class": "RetrievalHeadPyramidBottomUpProgressiveConv",
        "in_feat_idxs": [0, 1, 2, 3],
        "emb_sizes": [820, 956, 1024],
        "conv_par_perc": 0
    },
    "head_005": {
        "class": "RetrievalHeadPyramidBottomUpProgressiveConv",
        "in_feat_idxs": [0, 1, 2, 3],
        "emb_sizes": [512, 768, 1024],
        "conv_par_perc": 0
    },
    "head_006": {
        "class": "RetrievalHeadPyramidTopDownInstantSimple",
        "in_feat_idxs": [0, 1, 2, 3],
        "emb_size": 1024
    },
    "head_007": {
        "class": "RetrievalHeadPyramidTopDownProgressiveSimple",
        "in_feat_idxs": [0, 1, 2, 3],
        "emb_sizes": [205, 478, 1024]
    },
    "head_008": {
        "class": "RetrievalHeadPyramidTopDownProgressiveSimple",
        "in_feat_idxs": [0, 1, 2, 3],
        "emb_sizes": [512, 768, 1024]
    },
    "head_009": {
        "class": "RetrievalHeadPyramidTopDownInstantConv",
        "in_feat_idxs": [0, 1, 2, 3],
        "emb_size": 1024,
        "conv_par_perc": 0
    },
    "head_010": {
        "class": "RetrievalHeadPyramidTopDownProgressiveConv",
        "in_feat_idxs": [0, 1, 2, 3],
        "emb_sizes": [205, 478, 1024],
        "conv_par_perc": 0
    },
    "head_011": {
        "class": "RetrievalHeadPyramidTopDownProgressiveConv",
        "in_feat_idxs": [0, 1, 2, 3],
        "emb_sizes": [512, 768, 1024],
        "conv_par_perc": 0
    }
}

In [10]:
params_llist = [
    [1.78e-5, "head_001", "exp_006"],
    [3.16e-5, "head_001", "exp_007"],
    [5.62e-5, "head_001", "exp_008"],

    [1.78e-5, "head_002", "exp_006"],
    [3.16e-5, "head_002", "exp_007"],
    [5.62e-5, "head_002", "exp_008"],

    [1.78e-5, "head_003", "exp_006"],
    [3.16e-5, "head_003", "exp_007"],
    [5.62e-5, "head_003", "exp_008"],

    [1.78e-5, "head_004", "exp_006"],
    [3.16e-5, "head_004", "exp_007"],
    [5.62e-5, "head_004", "exp_008"],

    [1.78e-5, "head_005", "exp_006"],
    [3.16e-5, "head_005", "exp_007"],
    [5.62e-5, "head_005", "exp_008"],

    [1.78e-5, "head_007", "exp_006"],
    [3.16e-5, "head_007", "exp_007"],
    [5.62e-5, "head_007", "exp_008"],

    [1.78e-5, "head_008", "exp_006"],
    [3.16e-5, "head_008", "exp_007"],
    [5.62e-5, "head_008", "exp_008"],

    [1.78e-5, "head_009", "exp_006"],
    [3.16e-5, "head_009", "exp_007"],
    [5.62e-5, "head_009", "exp_008"],

    [1.78e-5, "head_010", "exp_006"],
    [3.16e-5, "head_010", "exp_007"],
    [5.62e-5, "head_010", "exp_008"],

    [1.78e-5, "head_011", "exp_006"],
    [3.16e-5, "head_011", "exp_007"],
    [5.62e-5, "head_011", "exp_008"],
]

for params_list in params_llist:

    lr = params_list[0]
    head_name = params_list[1]
    exp_name = params_list[2]

    head_num = int(head_name[5:])
    if head_num >= 7:
        exp_params["settings"]["device_idxs"] = [2, 4, 5, 6]
    else:
        exp_params["settings"]["device_idxs"] = [3, 7, 8, 9]

    full_exp_name = "{:s}_{:s}".format(head_name, exp_name)

    exp_params = (exp_params_template).copy()
    exp_params["experiment_name"] += full_exp_name
    exp_params["settings"]["stage_1"]["optimizer"]["lr"] = lr
    exp_params["settings"]["head"] = head_dicts[head_name]

    exp_params_filename = os.path.join("params", "ret_train_old_smpl", "convnext_v2_t_s1", full_exp_name + "__train_ret_DDP_s1.json")

    src.utils.json.save_json_dict(exp_params, exp_params_filename)

In [11]:
for params_list in params_llist:

    lr = params_list[0]
    head_name = params_list[1]
    exp_name = params_list[2]

    head_num = int(head_name[5:])
    if head_num >= 7:
        continue

    full_exp_name = "{:s}_{:s}".format(head_name, exp_name)
    exp_params_filename = os.path.join("params", "ret_train_old_smpl", "convnext_v2_t_s1", full_exp_name + "__train_ret_DDP_s1.json")

    command = "python train_ret_DDP_s1_NEW.py {:s} --master_port 12355 --terminal_silent --resume_experiment".format(exp_params_filename)

    print(command)

python train_ret_DDP_s1_NEW.py params/ret_train_old_smpl/convnext_v2_t_s1/head_001_exp_006__train_ret_DDP_s1.json --master_port 12355 --terminal_silent --resume_experiment
python train_ret_DDP_s1_NEW.py params/ret_train_old_smpl/convnext_v2_t_s1/head_001_exp_007__train_ret_DDP_s1.json --master_port 12355 --terminal_silent --resume_experiment
python train_ret_DDP_s1_NEW.py params/ret_train_old_smpl/convnext_v2_t_s1/head_001_exp_008__train_ret_DDP_s1.json --master_port 12355 --terminal_silent --resume_experiment
python train_ret_DDP_s1_NEW.py params/ret_train_old_smpl/convnext_v2_t_s1/head_002_exp_006__train_ret_DDP_s1.json --master_port 12355 --terminal_silent --resume_experiment
python train_ret_DDP_s1_NEW.py params/ret_train_old_smpl/convnext_v2_t_s1/head_002_exp_007__train_ret_DDP_s1.json --master_port 12355 --terminal_silent --resume_experiment
python train_ret_DDP_s1_NEW.py params/ret_train_old_smpl/convnext_v2_t_s1/head_002_exp_008__train_ret_DDP_s1.json --master_port 12355 --term

In [12]:
for params_list in params_llist:

    lr = params_list[0]
    head_name = params_list[1]
    exp_name = params_list[2]

    head_num = int(head_name[5:])
    if head_num < 7:
        continue

    full_exp_name = "{:s}_{:s}".format(head_name, exp_name)
    exp_params_filename = os.path.join("params", "ret_train_old_smpl", "convnext_v2_t_s1", full_exp_name + "__train_ret_DDP_s1.json")

    command = "python train_ret_DDP_s1_NEW.py {:s} --master_port 12356 --terminal_silent --resume_experiment".format(exp_params_filename)

    print(command)

python train_ret_DDP_s1_NEW.py params/ret_train_old_smpl/convnext_v2_t_s1/head_007_exp_006__train_ret_DDP_s1.json --master_port 12356 --terminal_silent --resume_experiment
python train_ret_DDP_s1_NEW.py params/ret_train_old_smpl/convnext_v2_t_s1/head_007_exp_007__train_ret_DDP_s1.json --master_port 12356 --terminal_silent --resume_experiment
python train_ret_DDP_s1_NEW.py params/ret_train_old_smpl/convnext_v2_t_s1/head_007_exp_008__train_ret_DDP_s1.json --master_port 12356 --terminal_silent --resume_experiment
python train_ret_DDP_s1_NEW.py params/ret_train_old_smpl/convnext_v2_t_s1/head_008_exp_006__train_ret_DDP_s1.json --master_port 12356 --terminal_silent --resume_experiment
python train_ret_DDP_s1_NEW.py params/ret_train_old_smpl/convnext_v2_t_s1/head_008_exp_007__train_ret_DDP_s1.json --master_port 12356 --terminal_silent --resume_experiment
python train_ret_DDP_s1_NEW.py params/ret_train_old_smpl/convnext_v2_t_s1/head_008_exp_008__train_ret_DDP_s1.json --master_port 12356 --term